In [1]:
import os
import cv2 as cv

images_path = r'.\data\png_images'
masks_path = r'.\data\png_masks'
processed_path = r'.\data\processed'

In [2]:
def resize_image(image, width=512, height=512):
    return cv.resize(image, (width, height))

def convert_to_grayscale(image):
    if len(image.shape) == 3 and image.shape[2] == 3:
        return cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    return image

def normalize_image(image):
    return cv.normalize(image, None, 0, 255, cv.NORM_MINMAX)

In [3]:
for file in os.listdir(images_path):
    file_path = os.path.join(images_path, file)
    image = cv.imread(file_path)
    
    if image is None:
        print(f"Warning: Unable to read image {file_path}. Skipping...")
        continue
    
    # Chia tên file để lấy id và nhãn
    file_parts = file.split('_')
    if len(file_parts) < 3:
        print(f"Warning: Invalid filename format {file}. Skipping...")
        continue
    
    i = file_parts[0]  # ID
    j = file_parts[2]  # Nhãn (0 hoặc 1)
    
    # Xác định thư mục lưu trữ dựa trên nhãn
    label_folder = 'has_pneumo' if j == '1' else 'normal'
    
    # Xác định kiểu dữ liệu là train hay test từ tên file
    data_type = 'train' if 'train' in file else 'test'
    
    # Tạo đường dẫn cho thư mục đã xử lý
    images_output_path = os.path.join(processed_path, 'images', label_folder, data_type)
    masks_output_path = os.path.join(processed_path, 'mask', label_folder, data_type)
    
    # Tạo các thư mục nếu chưa tồn tại
    os.makedirs(images_output_path, exist_ok=True)
    os.makedirs(masks_output_path, exist_ok=True)
    
    # Resize và chuẩn hóa ảnh
    resized_image = resize_image(image)
    normalized_image = normalize_image(convert_to_grayscale(resized_image))
    
    # Lưu ảnh đã xử lý vào thư mục tương ứng
    aug_file_name = f"{i}_train_{j}.png" if 'train' in file else f"{i}_test_{j}.png"
    cv.imwrite(os.path.join(images_output_path, aug_file_name), normalized_image)
    
    # Lưu mask nếu có
    mask_file_path = os.path.join(masks_path, file)  # Đường dẫn đến mask tương ứng
    mask_image = cv.imread(mask_file_path)
    
    if mask_image is not None:
        resized_mask = resize_image(mask_image)
        normalized_mask = normalize_image(resized_mask)
        cv.imwrite(os.path.join(masks_output_path, aug_file_name), normalized_mask)
    else:
        print(f"Warning: Unable to read mask for {file_path}. Skipping...")